# Lọc các bài báo có mang nội dung "lừa đảo"

## Khai báo thư viện

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

import time
import os
import json

## Set up môi trường seleniun

In [3]:
chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")
chrome_options.page_load_strategy = "eager"
MAGAZINE_NAME = "vnexpress"
HOME_PAGE = "https://timkiem.vnexpress.net/?q=L%E1%BB%ABa%20%C4%91%E1%BA%A3o"
driver = webdriver.Chrome(options=chrome_options)
driver.get(HOME_PAGE)

## Lọc ra các url

In [4]:
crawled_urls = set()

# Giới hạn số trang
max_pages = 39
current_page = 1

while current_page <= max_pages:
    # Tìm phần tử chứa kết quả tìm kiếm
    articles = driver.find_element(By.ID, 'result_search')

    # Tìm tất cả các thẻ h3 trong phần tử chứa kết quả
    h3_elements = articles.find_elements(By.TAG_NAME, 'h3')

    # Duyệt qua từng thẻ h3 để lấy các link
    for h3 in h3_elements:
        a_elements = h3.find_elements(By.TAG_NAME, 'a')
        for a in a_elements:
            href = a.get_attribute('href')
            if href:  # Đảm bảo href không rỗng
                crawled_urls.add(href)

    # Tìm "Next Page"
    try:
        url = driver.find_element(by=By.CLASS_NAME, value="next-page").get_attribute("href")
        driver.get(url)
        current_page += 1
    except Exception as e:
        print("Đã đọc đến page cuối cùng")
        break
driver.close()

Đã đọc đến page cuối cùng


### Lấy decription + content của các bài báo

In [5]:
def get_content_metadata(driver, article_url):

    # Get to current article
    driver.get(article_url)

    # Thu thập title
    title = driver.find_element(by=By.CSS_SELECTOR, value="h1.title-detail").text.strip()

    # Thu thập description
    description = driver.find_element(by=By.CLASS_NAME, value="description").text.strip()

    # Thu thập content bài báo
    # Locate phần viết content
    article = driver.find_element(by=By.CSS_SELECTOR, value="article.fck_detail")
    # Lấy hết các đầu mục con của bài báo
    children = article.find_elements(by=By.XPATH, value="./*")

    contents = []

    # Check có phải dạng slide show hay không
    is_slide_show = False
    for idx, child in enumerate(children):
        text = child.text.strip()
        # Nếu mà element right align --> có thể là tác giả
        if child.tag_name == "p" and ("right" in child.get_attribute("align") or "right" in child.get_attribute("style")) and idx >= len(children) - 3: # last three, align right --> author
            continue
        elif child.tag_name == "p" and child.get_attribute("class") == "Normal": # paragraph
            # If center
            if len(text):
                if ("center" in child.get_attribute("align") or "center" in child.get_attribute("style")):
                    contents.append(f"[{text}]")
                else:
                    contents.append(text)

        # Chỉ lấy caption của figure
        elif child.tag_name == "figure" :
            ## If length > 100  --> not a caption, it's next description
            if len(text):
                if len(text) <= 100: # nếu mà len <= 100 --> add thêm [] xung quanh
                    contents.append(f"[{text}]")
                else:
                    contents.append(text)

        # Nếu mà là slide show thì nó giống figure
        elif child.tag_name == "div" and "item_slide_show" in child.get_attribute("class"):
            is_slide_show = True # slideshow
            if len(text):
                if len(text) <= 100:
                    contents.append(f"[{text}]")
                else:
                    contents.append(text)

        # Bỏ qua table bây giờ
        elif child.tag_name == "table": # Do nothing rightnow
            pass

    return {
        "url": article_url,
        "title": title,
        "description": description,
        "content": "\n".join(contents), # join các đoạn bằng \n
    }

In [6]:
driver = webdriver.Chrome(options=chrome_options)
dataset = []
urls_to_crawl = list(crawled_urls)  # Tạo bản sao của danh sách

for url in urls_to_crawl:
    try:
        dataset.append(get_content_metadata(driver, url))
    except (StaleElementReferenceException, NoSuchElementException) as e:
        print(f"Bug at url: {url}, with ElementException")
        # driver.refresh()
        crawled_urls.remove(url)
        continue
with open('dataset.json', 'w', encoding="utf-8") as file:
    json.dump(list(dataset), file, ensure_ascii=False, indent=4)


Bug at url: https://vnexpress.net/bay-hen-ho-tuoi-trung-nien-4775437.html, with ElementException
Bug at url: https://vnexpress.net/vu-gian-lan-ke-toan-thiet-hai-khung-nhat-nuoc-my-4732126.html, with ElementException
Bug at url: https://vnexpress.net/chieu-lua-dat-don-hang-ao-4497269.html, with ElementException
Bug at url: https://vnexpress.net/sach-tai-khoan-vi-sap-bay-shipper-gia-mao-4765437.html, with ElementException
Bug at url: https://vnexpress.net/facebook-bi-kien-vi-quang-cao-lua-dao-tien-dien-tu-4441228.html, with ElementException
Bug at url: https://vnexpress.net/ben-trong-lo-dao-tao-lua-dao-4529922.html, with ElementException
Bug at url: https://video.vnexpress.net/so-hoa/lua-ao-qua-webcam-1791631.html, with ElementException
Bug at url: https://video.vnexpress.net/tin-tuc/phap-luat/nguoi-dan-ba-bi-dieu-tra-lua-33-ty-dong-4360015.html, with ElementException
Bug at url: https://vnexpress.net/dai-gia-o-sai-gon-bi-lua-88-ty-dong-4676149.html, with ElementException
Bug at url: htt